In [1]:
from tremlib import TremResearcher
import pandas as pd
import numpy as np
import os, glob


In [37]:
colors = ['Красный', 'Желтый', 'Зеленый', 'Синий', 'Белый', 'Черный']
write_type = ['Глаза открыты 1', 'Глаза открыты 2', 'Глаза закрыты 1', 'Глаза закрыты 2']

researcher = TremResearcher(marks_num=5, marks_size_mm=10)

In [38]:
def video_files(path, sort=True):
    d = os.path.abspath(os.curdir)
    os.chdir(path)
    l = ["%s\\%s" % (path, file) for file in glob.glob("*.mp4")]
    os.chdir(d)
    return sorted(l) if sort else l


def get_subdirs(path, sort=True):
    subdirs = []
    for file in os.listdir(path):
        if os.path.isdir(os.path.abspath(path)+'\\'+file):
            subdirs.append("%s\\%s" % (path, file))
    return sorted(subdirs) if sort else subdirs


def save_excel(df, sv_dir, file_name):
    if not os.path.isdir(sv_dir):
        os.makedirs(sv_dir)
    df.to_excel('%s//%s.xlsx' % (sv_dir, file_name))


In [ ]:
%%time



reports, raw_reports = [], []


subdirs = get_subdirs('.\\raw_videos')

#subdirs.remove('.\\Отчеты\\Берестин Дмитрий Констанинович')

for experiment_num, subdir in enumerate(subdirs):
    #!!!!!TODO УБРАТЬ!!!!!
    if subdir != '.\\raw_videos\\Бурыкин Юрий Геннадьевич':
        continue
        
    print(subdir, "- detection started (%2d/%2d)" % (experiment_num, len(subdirs)))
    videos = video_files(subdir)
    for i, video in enumerate(videos):
        fio = subdir.split('\\')[-1]
        uni_file_name = '%s_%s' % (write_type[i//len(colors)], colors[i % len(colors)])
        raw_report, square_sizes = researcher.get_raw_report(
            file_path=video, out_video=True,
            path_to_save='.\\Отчеты\\%s\\Размеченные видео' % fio, out_video_name=uni_file_name)
        
        if raw_report is not None:
            df_report = researcher.norm_report(raw_report, square_sizes)
            reports.append(df_report)
            raw_reports.append(raw_report)            
            save_excel(df_report, '.\\Отчеты\\%s\\Excel' % fio, uni_file_name)
            
            researcher.draw_separate(df_report, uni_file_name, '.\\Отчеты\\%s\\Графики' % fio)
            researcher.draw_together(raw_report, uni_file_name, '.\\Отчеты\\%s\\Графики' % fio)
        else:
            print("FILE: %s has no markers" % video)
        print(video, "- detection finished  (%3.2f%%)" % ((i+1)/len(videos)*100) )
print("WORK DONE")


.\raw_videos\Бурыкин Юрий Геннадьевич - detection started ( 2/ 5)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0664.MP4 - detection finished  (4.17%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0665.MP4 - detection finished  (8.33%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0666.MP4 - detection finished  (12.50%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0667.MP4 - detection finished  (16.67%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0668.MP4 - detection finished  (20.83%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0669.MP4 - detection finished  (25.00%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0670.MP4 - detection finished  (29.17%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0671.MP4 - detection finished  (33.33%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0672.MP4 - detection finished  (37.50%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0673.MP4 - detection finished  (41.67%)
.\raw_videos\Бурыкин Юрий Геннадьевич\SAM_0674.MP4 - detection finished  (45.83%)
.\raw_videos\Бурыкин Юрий Геннадье

In [39]:
def full_report(video_path):
    if "\\" in video_path:
         path, video = video_path.rsplit("\\", 1)
    else:
         path, video = '.\\', video_path
    name, ext = video.rsplit('.', 1)
    raw_report, square_sizes = researcher.get_raw_report(
        file_path=video_path, out_video=True,
        path_to_save=path, out_video_name="%s (размечено).%s" % (name, ext))
    print(raw_report, square_sizes)
    df_report = researcher.norm_report(raw_report, square_sizes)
    if df_report is not None:
        save_excel(df_report, path, name)

        researcher.draw_separate(df_report, name, path)
        researcher.draw_together(raw_report, name, path)
    else:
        print("FILE: %s has no markers" % video)
    return df_report

In [41]:
videos = video_files(r'F:\Projects\Tremor\Проверка фокуса 50мм')
for i, video_path in enumerate(videos):
    rep = full_report(video_path)
    print(video_path, "- detection finished  (%3.2f%%)" % ((i+1)/len(videos)*100) )


№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1171.281738  588.366577  1018.153076  413.096893  682.578003   
20.0       1171.686035  587.947815  1018.320984  412.739563  682.488220   
40.0       1172.450073  587.295349  1018.379883  412.054047  682.588684   
60.0       1172.970947  587.729858  1018.353088  412.208710  682.374329   
80.0       1173.257324  588.593933  1018.029297  412.634827  682.291077   
100.0      1173.581787  589.668213  1017.700073  413.256470  682.013184   
120.0      1173.745728  589.676636  1017.622620  413.184631  681.761292   
140.0      1173.823975  589.147034  1017.435791  412.529236  682.200256   
160.0      1173.814575  589.184082  1017.276428  412.605164  681.729858   
180.0      1174.126709  589.983521  1017.290283  413.355865  681.696899   
200.0      1174.465698  5

F:\Projects\Tremor\Проверка фокуса 50мм\1Б.MP4 - detection finished  (8.33%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1144.566284  531.703186  1005.996582  411.978027  684.762146   
20.0       1144.429565  531.620972  1005.725464  411.634918  684.487183   
40.0       1144.709229  531.396851  1005.752319  410.985565  684.508911   
60.0       1144.815063  531.205261  1005.864502  410.878784  684.769043   
80.0       1145.079346  531.700378  1005.844482  411.407471  684.742188   
100.0      1145.278809  532.811584  1006.132935  412.548523  684.801636   
120.0      1145.339111  533.771301  1006.076843  413.238403  684.609924   
140.0      1145.094238  533.666687  1005.773438  413.295807  684.581909   
160.0      1145.046143  533.124390  1005.919312  412.889160  684.766724   
180.0      1144.995117 

F:\Projects\Tremor\Проверка фокуса 50мм\1Ж.MP4 - detection finished  (16.67%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1141.920288  548.244751   998.284424  407.663635  689.428650   
20.0       1141.761475  548.232422   998.136658  407.261353  689.398560   
40.0       1141.683228  547.647888   998.355652  406.738525  689.155762   
60.0       1141.534058  547.391785   998.278564  406.747314  688.768921   
80.0       1141.753418  548.217651   998.506104  407.163025  689.045349   
100.0      1141.993896  549.346802   998.544739  407.923340  689.022827   
120.0      1142.311035  549.510864   998.490356  407.673340  688.760681   
140.0      1142.074463  548.818604   998.354553  407.230164  688.474854   
160.0      1141.699585  548.107666   998.150757  406.951324  688.254456   
180.0      1141.770508

F:\Projects\Tremor\Проверка фокуса 50мм\1З.MP4 - detection finished  (25.00%)
№ метки              1                       2                       3  \
Ось                  X           Y           X           Y           X   
Время, мс                                                                
0.0        1155.027100  507.767395  967.219116  350.703827  633.242249   
20.0       1154.957886  507.690735  967.052979  350.517487  633.019409   
40.0       1155.130371  508.669556  967.300781  351.242554  633.263184   
60.0       1155.447876  509.722351  968.049561  352.637695  633.776917   
80.0       1155.775879  509.969757  968.199463  353.528412  633.993774   
100.0      1155.763184  509.974792  968.293701  353.903931  633.804443   
120.0      1155.753174  509.846924  967.585693  353.326202  633.251587   
140.0      1156.064697  509.842712  967.614563  352.774963  633.141357   
160.0      1156.096680  510.237976  968.001404  352.999756  633.336365   
180.0      1155.774658  510.945282

F:\Projects\Tremor\Проверка фокуса 50мм\1К.MP4 - detection finished  (33.33%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1150.590088  556.268555  1021.744629  382.596130  698.520264   
20.0       1150.516357  555.947327  1021.521118  381.890015  698.451782   
40.0       1150.483276  555.835144  1021.458557  381.639648  698.643433   
60.0       1150.800903  555.829712  1021.512695  380.816559  698.751282   
80.0       1151.160400  556.138123  1021.749207  381.301239  698.663025   
100.0      1151.635132  557.082031  1021.821350  382.552765  698.574768   
120.0      1151.854736  558.375732  1021.693848  383.605957  698.334595   
140.0      1151.802246  559.508057  1021.364502  383.989380  698.113281   
160.0      1152.276489  559.687744  1021.479736  383.880798  698.118713   
180.0      1152.794678

...

F:\Projects\Tremor\Проверка фокуса 50мм\2З.MP4 - detection finished  (75.00%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1264.004150  513.671753   999.496216  470.155701  672.228821   
20.0       1263.803223  513.780640   999.585205  470.145691  672.109619   
40.0       1263.797607  513.671570   999.422363  469.620483  672.071838   
60.0       1263.504883  514.316101   999.143799  469.679474  672.008301   
80.0       1262.960205  516.020996   998.978394  470.523590  671.917480   
100.0      1262.597412  516.750183   998.950073  471.375092  671.929626   
120.0      1262.828857  516.731812   998.931091  471.699982  671.970459   
140.0      1263.026123  516.510559   998.964905  471.367126  671.928284   
160.0      1263.032959  516.763977   999.068726  471.222656  671.950806   
180.0      1262.847168

F:\Projects\Tremor\Проверка фокуса 50мм\2К.MP4 - detection finished  (83.33%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1230.830444  523.852539  1149.283447  397.405670  822.961243   
20.0       1230.771362  523.589783  1149.583252  396.663605  823.420044   
40.0       1230.962646  524.155640  1149.703125  397.307434  823.460999   
60.0       1231.003784  525.120911  1149.623047  398.240845  823.396484   
80.0       1230.901367  525.758301  1149.521240  398.233765  823.161072   
100.0      1231.397095  525.063538  1149.484375  397.556580  823.216919   
120.0      1231.387573  524.389709  1149.562988  396.580872  823.211487   
140.0      1231.212280  524.399414  1149.625732  396.518555  823.073303   
160.0      1231.036133  525.269043  1149.630249  397.260315  823.092041   
180.0      1231.387451

F:\Projects\Tremor\Проверка фокуса 50мм\2С.MP4 - detection finished  (91.67%)
№ метки              1                        2                       3  \
Ось                  X           Y            X           Y           X   
Время, мс                                                                 
0.0        1249.908447  529.600281  1087.913818  396.871735  811.729065   
20.0       1249.940918  530.866699  1087.967773  398.749451  811.572632   
40.0       1249.518677  531.949585  1087.855225  400.589783  811.853027   
60.0       1248.471680  532.066650  1087.811768  401.187134  811.583191   
80.0       1247.630981  532.367249  1087.298340  400.948547  810.760437   
100.0      1246.980225  533.169312  1086.897827  400.962341  810.432129   
120.0      1247.253784  533.125732  1086.736206  400.589478  810.442200   
140.0      1247.391724  532.711304  1086.624146  400.325897  810.687866   
160.0      1247.764038  531.635132  1087.090088  399.568817  811.186523   
180.0      1248.169189

F:\Projects\Tremor\Проверка фокуса 50мм\2Ч.MP4 - detection finished  (100.00%)


<Figure size 432x288 with 0 Axes>

**Пробуем откалибровать камеру**

In [ ]:
def photo_files(path, sort=True):
    d = os.path.abspath(os.curdir)
    os.chdir(path)
    l = ["%s\\%s" % (path, file) for file in glob.glob("*.jpg")]
    os.chdir(d)
    return sorted(l) if sort else l

for photo in photo_files('.\\'):


In [ ]:
from multiprocess import Pool, TimeoutError
import time
import os

def f(x):
    return x*x

if __name__ == '__main__':
    # start 4 worker processes
    with Pool() as pool:

        # print "[0, 1, 4,..., 81]"
        print(pool.map(f, range(10)))

        # print same numbers in arbitrary order
        for i in pool.imap_unordered(f, range(10)):
            print(i)

        # evaluate "f(20)" asynchronously
        res = pool.apply_async(f, (20,))      # runs in *only* one process
        print(res.get(timeout=1))             # prints "400"

        # evaluate "os.getpid()" asynchronously
        res = pool.apply_async(os.getpid, ()) # runs in *only* one process
        print(res.get(timeout=1))             # prints the PID of that process

        # launching multiple evaluations asynchronously *may* use more processes
        multiple_results = [pool.apply_async(os.getpid, ()) for i in range(4)]
        print([res.get(timeout=1) for res in multiple_results])

        # make a single worker sleep for 10 secs
        res = pool.apply_async(time.sleep, (10,))
        try:
            print(res.get(timeout=1))
        except TimeoutError:
            print("We lacked patience and got a multiprocessing.TimeoutError")

        print("For the moment, the pool remains available for more work")

    # exiting the 'with'-block has stopped the pool
    print("Now the pool is closed and no longer available")